# Assignment 3: Improving the Index

In this assignment, we will improve the search index and query functions from the previous assignment.

## Loading the Data and Defining Auxiliary Functions

This section is copied from the previous notebook.

In [2]:
import pickle, bz2, re
from collections import namedtuple, defaultdict, Counter
from IPython.display import display, HTML
from math import log10, sqrt

Summaries_file = 'data/recognition_Summaries.pkl.bz2'
Abstracts_file = 'data/recognition_Abstracts.pkl.bz2'

Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )
Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )
for (id, paper_info) in Summaries.items():
    Summaries[id] = paper( *paper_info )

In [3]:
def tokenize(text):
    """
    Function that tokenizes a string in a rather naive way. Can be extended later.
    """
    return text.split(' ')

def preprocess(tokens):
    """
    Perform linguistic preprocessing on a list of tokens. Can be extended later.
    """
    result = []
    for token in tokens:
        result.append(token.lower())
    return result

In [4]:
def display_summary( id, show_abstract=False, show_id=True, extra_text='' ):
    """
    Function for printing a paper's summary through IPython's Rich Display System.
    Trims long author lists, and adds a link to the paper's DOI (when available).
    """
    s = Summaries[id]
    lines = []
    title = s.title
    if s.doi != '':
        title = '<a href=http://dx.doi.org/{:s}>{:s}</a>'.format(s.doi, title)
    title = '<strong>' + title + '</strong>'
    lines.append(title)
    authors = ', '.join( s.authors[:20] ) + ('' if len(s.authors) <= 20 else ', ...')
    lines.append(str(s.year) + '. ' + authors)
    if (show_abstract):
        lines.append('<small><strong>Abstract:</strong> <em>{:s}</em></small>'.format(Abstracts[id]))
    if (show_id):
        lines.append('[ID: {:d}]'.format(id))
    if (extra_text != ''):
         lines.append(extra_text)
    display( HTML('<br>'.join(lines)) )

In [5]:
inverted_index = defaultdict(list)

for id in sorted(Summaries.keys()):
    term_set = set(preprocess(tokenize(Summaries[id].title)))
    if id in Abstracts:
        term_set.update(preprocess(tokenize(Abstracts[id])))
    for term in term_set:
        inverted_index[term].append(id)
    


## Stemming

As we could see from the results of the last assignment, our simple index doesn't handle punctuation and the difference between singular and plural versions of the same word very well. We won't go much into the details of tokenization and linguistic analysis here, because we also want to focus on scoring and ranking below. Therefore, we are using an existing library for tokenizatoin and stemming, namely the NLTK package. The following line will install NLTK if necessary (or you have to follow [these instructions](http://www.nltk.org/install.html) if that doesn't work):

In [6]:
! pip install --user nltk

In [7]:
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import EnglishStemmer
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')
stemmer = EnglishStemmer()

s = '''Good muffins cost $4.88\nin New York.  Please buy me two of them.\n\nThanks.'''

print('INPUT TEXT:\n ', s)

print('TOKENIZE: ', tokenize(s))
print('WORD TOKENIZE: ', word_tokenize(s))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dbach\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dbach\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dbach\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\dbach\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


INPUT TEXT:
  Good muffins cost $4.88
in New York.  Please buy me two of them.

Thanks.
TOKENIZE:  ['Good', 'muffins', 'cost', '$4.88\nin', 'New', 'York.', '', 'Please', 'buy', 'me', 'two', 'of', 'them.\n\nThanks.']
WORD TOKENIZE:  ['Good', 'muffins', 'cost', '$', '4.88', 'in', 'New', 'York', '.', 'Please', 'buy', 'me', 'two', 'of', 'them', '.', 'Thanks', '.']


In [8]:
print(stemmer.stem("processes"))

process


## Ranking

Another important method to improve our search results is to rank them, which can be done by calculating a score for each document based on the matching terms from the query. One such scoring method is *tf-idf*, which comes with several variants, as explained in the lecture slides.

In order to quickly calculate the scores for a term/document combination, we'll need quick access to a couple of things:

- tf(t,d): How often does a term occur in a document
- df(t): In how many documents does a term occur
- num_documents: The number of documents in our index

In [9]:
tf_matrix = defaultdict(Counter)

for doc_id in Summaries.keys():
    tokens = preprocess(tokenize(Summaries[doc_id].title))
    if (doc_id in Abstracts):
        tokens.extend(preprocess(tokenize(Abstracts[doc_id])))
    tf_matrix[doc_id] = Counter(tokens)

def tf(t,d):
    return float(tf_matrix[d][t])

def df(t):
    return float(len(inverted_index[t]))

num_documents = float(len(Summaries))

Let's test these functions with some examples:

In [10]:
print(tf('enzyme', 33055208))
print(df('enzyme'))
print(num_documents)

print(tf('rot', 33055208))
print(df('rot'))
print(num_documents)

1.0
11027.0
481216.0
0.0
80.0
481216.0


With these helper functions, we can now easily calculate the _tf-idf_ weights of a term in a document by implementing the weighting formula from the slides, which you will do in the assignments below.

----------

# Tasks

Dimitar Bachvarov

### Task 1

Implement in the code block below the `smart_tokenize_and_preprocess` function using NLTK's functions for tokenization and stemming. 

In [11]:
# Smarter linguistic processing
def smart_tokenize_and_preprocess(text):
    tokens = word_tokenize(text)
    result = []
    for T in tokens:
        result.append(stemmer.stem(T))
    return result


# To test it:
print(smart_tokenize_and_preprocess("Books about Information Retrieval (IR) etc. cost at least $25.00!"))

['book', 'about', 'inform', 'retriev', '(', 'ir', ')', 'etc', '.', 'cost', 'at', 'least', '$', '25.00', '!']


Now we can make a smarter index based on this function. For practical purposes, the code below generates the smarter index on a subset of the data, as generating an index with stemming on the entire set would take too much time. (You don't need to change or add anything in the code block below. Just leave it as it is.)

In [12]:
# Below, we create our smarter index (based on a subset of the documents for demonstration purposes)
smarter_index = defaultdict(list)

# Here we define the subset (somewhat arbitrary):
subset_of_ids = list(key for key in Summaries.keys() if 33000000 <= key < 34000000)

# Building our smarter index:
for id in sorted(subset_of_ids):
    term_set = set(smart_tokenize_and_preprocess(Summaries[id].title))
    if id in Abstracts:
        term_set.update(smart_tokenize_and_preprocess(Abstracts[id]))
    for term in term_set:
        smarter_index[term].append(id)

Now implement the `smarter_and_query` function, based on the function `smarter_tokenize_and_preprocess` you defined above and accessing our new index `smarter_index`. You can start from copying the code for `and_query` from the last assignment. For that to work, you'll also have to copy the code for the `and_merge` function from the last assignment.

In [13]:
# Smarter and_query based on the smarter tokenize and preprocess functions

list1 = [3,5,8,11]
list2 = [5,7,8,9,11,12]
def merge_sorted_and(l1 : list, l2 : list):
    positionl1 : int = 0
    positionl2 : int = 0
    currValueL1 : int = l1[positionl1]
    currValueL2 : int = l2[positionl2]
    resultList : list = []

    while (positionl1 < len(l1) and positionl2 < len(l2)):
        currValueL1 = l1[positionl1]
        currValueL2 = l2[positionl2]
        if currValueL1 < currValueL2:
            positionl1 = positionl1 + 1
        elif currValueL1 > currValueL2:
            positionl2 = positionl2 + 1
        else: 
            resultList.append(l1[positionl1])
            positionl1 = positionl1 + 1
            positionl2 = positionl2 + 1
            
    return resultList

merge_sorted_and(list1, list2)

quary = 'music guitar dataset'
def smarter_and_query(quary : str):
   matchingDocList = []
   wordList = smart_tokenize_and_preprocess(quary)
   print(wordList)
   for word in wordList:
      matchingDocList.append(smarter_index[word])
      print(word, ": ", smarter_index[word])

   result = matchingDocList[0]
   
   if len(matchingDocList) <= 1 and len(matchingDocList) >= 0:
      return result
   elif len(matchingDocList) > 1:
      for list in matchingDocList:
            result = merge_sorted_and(result, list)
      return result

result = smarter_and_query(quary)
print('Result:', result)
# 28450846 for guitar not found!

['music', 'guitar', 'dataset']
music :  [33032499, 33046161, 33049954, 33074236, 33078363, 33090055, 33114599, 33141730, 33201423, 33254374, 33281705, 33321542, 33398938, 33448253, 33469435, 33511150, 33568976, 33602997, 33617124, 33647221, 33660560, 33679539, 33694052, 33705316, 33735039, 33776661, 33784471, 33808989, 33846254, 33868123, 33881610, 33912101, 33945127, 33947029, 33954279, 33989299]
guitar :  [33114599]
dataset :  [33003198, 33004380, 33013279, 33013674, 33014032, 33014181, 33014355, 33014638, 33014803, 33017173, 33017929, 33017942, 33018071, 33018073, 33018079, 33018089, 33018094, 33018153, 33018186, 33018255, 33018719, 33018857, 33018915, 33019205, 33019221, 33019238, 33019243, 33019248, 33019284, 33019293, 33019304, 33022966, 33028812, 33028972, 33034326, 33035162, 33036479, 33039785, 33039787, 33039979, 33041409, 33043311, 33050164, 33052737, 33053720, 33055923, 33059236, 33066123, 33066285, 33067452, 33068261, 33068806, 33074807, 33075638, 33076258, 33078363, 330788

### Task 2

Run the query "mutation of enzyme complexes" with the new `smarter_and_query` function from task 1. Does it return paper *33055208*? Explain what our new smarter function specifically contributes to the result (as compared to our previous naive implementations for tokenization and preprocessing)?

In [14]:
result = smarter_and_query("mutation of enzyme complexes")
print('Result:', result)
display_summary(33055208, show_abstract=True)

['mutat', 'of', 'enzym', 'complex']
mutat :  [33000558, 33004795, 33006091, 33009465, 33014341, 33014620, 33020180, 33023230, 33029712, 33031103, 33035266, 33037049, 33040723, 33045225, 33048983, 33050534, 33051095, 33051185, 33052530, 33052948, 33053321, 33053818, 33053912, 33054048, 33055208, 33056978, 33057152, 33057400, 33058871, 33058872, 33058888, 33060564, 33063358, 33064343, 33064779, 33066313, 33068046, 33068261, 33070870, 33087132, 33089614, 33090493, 33092221, 33099813, 33101452, 33104793, 33107822, 33108877, 33109023, 33109150, 33109263, 33119686, 33120458, 33122294, 33123120, 33125859, 33126897, 33127861, 33130828, 33138654, 33151007, 33152247, 33154402, 33158990, 33158995, 33159335, 33165832, 33165914, 33170304, 33171486, 33172890, 33175195, 33176178, 33189935, 33196685, 33197299, 33200028, 33201916, 33202035, 33202125, 33203705, 33203879, 33206170, 33208539, 33219211, 33221702, 33222322, 33225257, 33227812, 33229797, 33234888, 33235207, 33236215, 33244787, 33244925, 3325

**Answer:** By tokenizing words more accurately and applying stemming, we can retrieve papers that contain our query words without worrying about surrounding punctuation or slight variations in the words (examples: enzym -> enzyme -> enzymes / mutat -> mutation -> "mutations"). This will prevent the mistake that the past naive implamentation made which is excluding some document based on their variations. 

### Task 3

Now we move to a different subject and use our old index again. That is, we **don't** use the smarter functions defined above for tasks 3 to 5!

Create a function `tfidf(t,d)` that returns the tf-idf score of term `t` in document `d` by using `tf(t,d)`, `df(t)` and `num_documents` as defined above. To do this, first implement a function `idf(t)` to calculate the inverse document frequency, and then use this function to calculate the full tf-idf. Use the _add-one-smoothing_ version of idf, so we don't run into problems with terms that don't appear in the collection at all. The relevant formulas can be found on the lecture slides. Use tf-idf with plain (non-logarithmic) term frequency, as applied by scoring variant `ntn`. Test your function with the examples shown below. You can use the `log10(n)` function to calculate the base 10 logarithm.

Again, use our old (non-smart) index for this task and the tasks below, and **not** the functions defined in tasks 1 and 2.

In [15]:
# Your code here:
def idf(t):
    result_idf = log10((num_documents + 1)/(df(t) + 1))
    #print('result_idf:', result_idf)
    return result_idf #Inverse doc freq

def tfidf(t, d):
    result_tfidf = tf(t, d) * idf(t)
    #print('result_tfidf:', result_tfidf)
    return result_tfidf 

print(tfidf('mutation', 33055208))
print(tfidf('of', 33055208))
print(tfidf('enzyme', 33055208))
print(tfidf('complexes', 33055208))

0.0
0.14123984973343998
1.6398442038686944
0.0


### Task 4

Create a function `query_ntn_nnn(query_string)`, which accepts as input a single query string of one or more words, and returns or prints a list of (up to) 10 best matching documents, along with their score. Use _tf-idf_ to calculate document scores based on the query, applying variant `ntn.nnn`, as above (see the formula for the `ntn.nnn` version of scoring on the lecture slides). Use an auxiliary function `score_ntn_nnn` to calculate the score. The results should be shown in descending order by score.

You can start by copying your functions `or_merge` and `or_query` from assignment 2, then expand that to rank the results, making use of the `tfidf(t,d)` function you created above.

Demonstrate your function by giving it the exemplary query string "mutation of enzyme complexes".

In [16]:
def query_ntn_nnn(query_string):
    listOfDoc = or_query(query_string)
    scores = []
    for doc in listOfDoc:
       scores.append((score_ntn_nnn(query_string, doc), doc))
    top10_scores = sorted(scores, reverse = True)[:10]
    return top10_scores

def score_ntn_nnn(quary, doc):
    tokens = preprocess(tokenize(quary))
    sum = 0
    for word in tokens:
        sum = sum + tfidf(word, doc)
    return sum
    

def exhaustList(list : list, pos : int, result : list):
    while pos < len(list):
        result.append(list[pos])
        pos = pos + 1
    return result

def merge_sorted_or(l1 : list, l2 : list):
    positionl1 : int = 0
    positionl2 : int = 0
    currValueL1 : int = list1[positionl1]
    currValueL2 : int = list2[positionl2]
    resultList : list = []

    while (positionl1 < len(l1) and positionl2 < len(l2)):
        currValueL1 = l1[positionl1]
        currValueL2 = l2[positionl2]
        if currValueL1 < currValueL2:
            resultList.append(l1[positionl1])
            positionl1 = positionl1 + 1
        elif currValueL1 > currValueL2:
            resultList.append(l2[positionl2])
            positionl2 = positionl2 + 1
        else: 
            resultList.append(l1[positionl1])
            positionl1 = positionl1 + 1
            positionl2 = positionl2 + 1
        
    if positionl1 >= len(l1):
        resultList = exhaustList(l2, positionl2, resultList)
    elif positionl2 >= len(l2):
        resultList = exhaustList(l1, positionl1, resultList)

    return resultList

def or_query(quary : str):
   matchingDocList = []
   wordList = preprocess(tokenize(quary))
   print(wordList)
   for word in wordList:
      matchingDocList.append(inverted_index[word])
      #print(word, ": ", inverted_index[word])

   result = matchingDocList[0]
   
   if len(matchingDocList) <= 1 and len(matchingDocList) >= 0:
      return result
   elif len(matchingDocList) > 1:
      for list in matchingDocList:
            result = merge_sorted_or(result, list)
      return result


# Example query:
#query_ntn_nnn("mutation of enzyme complexes")
query_ntn_nnn("mutation of enzyme complexes")

['mutation', 'of', 'enzyme', 'complexes']


[(29.607075574012324, 39073830),
 (22.553644894234456, 17264471),
 (21.561934390741698, 7165718),
 (21.523414431723484, 10503276),
 (21.45921450002647, 16666772),
 (20.421708296058004, 1562739),
 (20.324470890964882, 28270751),
 (20.285950931946672, 26448454),
 (20.126113491841952, 11234382),
 (18.46776339578783, 11811950)]

### Task 5

In this last task, you should create a second version of the query function from Task 4, called `query_ntc_ntc`. This second version should use, as its name suggests, variant `ntc.ntc` instead of `ntn.nnn`, and therefore apply the cosine similarity measure, in addition to applying _tf-idf_. For this, consult the formula for variant `nnc.nnc` on the lecture slides and adopt it to include the _idf_ metric (that is, add the `t` element of `ntc`). (You can drop the square root of |q| in the formula, as indicated on the slides.)

As a first step, we can calculate beforehand the length of all document vectors (because they don't depend on the query) for document vectors consisting of _tf-idf_ values. The code below does just that, assuming that you defined the function `tfidf(t,d)` above (don't change this code block, just run it):

In [17]:
tfidf_length_values = defaultdict(int)

for doc_id in Summaries.keys():
    l = 0
    for t in tf_matrix[doc_id].keys():
        l += tfidf(t,doc_id) ** 2
    tfidf_length_values[doc_id] = sqrt(l)

def tfidf_length(d):
    return tfidf_length_values[d]

We can now get the length of a document vector by calling `tfidf_length(d)`.

Based on this, you can now implement `query_ntc_ntc` in the code block below. You should again first define an auxiliary function, called `score_ntc_ntc`. You can start by copy-pasting the code from Task 4.

To output the results, use the provided `display_summary` function to make the output a bit more like the results page of a search engine. Lastly, demonstrate your `query_ntc_ntc` function with the same example query as above: "mutation of enzyme complexes"

In [51]:
def query_ntc_ntc(query_string):
    listOfDoc = or_query(query_string)
    scores = []
    for doc in listOfDoc:
       scores.append((score_ntc_ntc(query_string, doc), doc))
    top10_scores = sorted(scores, reverse = True)[:10]
    print("top 10 score/docID tuples: " , top10_scores)
    docID = [x[1] for x in top10_scores]
    for x in docID:
        display_summary(x)
    return top10_scores

def score_ntc_ntc(quary, doc):
    tokens = preprocess(tokenize(quary))
    sum = 0
    for word in tokens:
        sum = sum + tfidf(word, doc)
    return sum / tfidf_length(doc)

# Example query:
query_ntc_ntc("mutation of enzyme complexes")

['mutation', 'of', 'enzyme', 'complexes']
top 10 score/docID tuples:  [(0.5530487619257687, 4052582), (0.532595598639559, 34513841), (0.5304478614481015, 39073830), (0.5082897381974015, 36342236), (0.4962501427925369, 17327177), (0.48486369982152844, 6967885), (0.46459749185662874, 8197478), (0.4492725176646092, 6185948), (0.4386895861376851, 10503276), (0.43824045520868055, 10084112)]


[(0.5530487619257687, 4052582),
 (0.532595598639559, 34513841),
 (0.5304478614481015, 39073830),
 (0.5082897381974015, 36342236),
 (0.4962501427925369, 17327177),
 (0.48486369982152844, 6967885),
 (0.46459749185662874, 8197478),
 (0.4492725176646092, 6185948),
 (0.4386895861376851, 10503276),
 (0.43824045520868055, 10084112)]

# Submission

Submit the answers to the assignment via Canvas as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers.

Before submitting, restart the kernel and re-run the complete code (**Kernel > Restart & Run All**), and then check whether your assignment code still works as expected.

Don't forget to add your name, and remember that the assignments have to be done **individually**, and that code sharing or copying are **strictly forbidden** and will be punished.